In [1]:
#import all helpers
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import statsmodels.api as sm
from patsy import dmatrices
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn import metrics
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV
import seaborn as sns # More snazzy plotting library
import itertools

#import regressors
#-----Ensemble--------------------- Kelvin
from sklearn.ensemble import       AdaBoostRegressor
from sklearn.ensemble import       BaggingRegressor
from sklearn.ensemble import       ExtraTreesRegressor
from sklearn.ensemble import       GradientBoostingRegressor
from sklearn.ensemble import       RandomForestRegressor

#----Generalized Linear models----- Su, Ronald
from sklearn.linear_model import   ARDRegression
from sklearn.linear_model import   BayesianRidge
from sklearn.linear_model import   ElasticNet
from sklearn.linear_model import   HuberRegressor
from sklearn.linear_model import   Lars
from sklearn.linear_model import   Lasso
from sklearn.linear_model import   LassoLars
from sklearn.linear_model import   LinearRegression
from sklearn.linear_model import   PassiveAggressiveRegressor
from sklearn.linear_model import   Ridge
from sklearn.linear_model import   SGDRegressor

#---Nearest Neighbors----
from sklearn.neighbors import      KNeighborsRegressor
from sklearn.neighbors import      RadiusNeighborsRegressor


#----Neural Networks--------------- Ahmet
from sklearn.neural_network import MLPRegressor

#-----Support Vector Machines------
from sklearn.svm import            SVR
from sklearn.svm import            LinearSVR
from sklearn.svm import            NuSVR

#-----Decission Trees-------------- Kelvin
from sklearn.tree import           DecisionTreeRegressor
from sklearn.tree import           ExtraTreeRegressor

#file_path =  "../dataset/movie_metadata_cleaned_tfidf_num_only_min.csv"
file_path =  "../dataset/movie_metadata_cleaned_categ_num_only.csv"
#file_path = "../dataset/movie_metadata_cleaned_no_vector_num_only.csv"

dta = pd.read_csv(file_path)
dta_clean = dta
#remove the null values, that is fill NaN with there - FIXME: Rihards, naive implementation
dta_clean = dta_clean.fillna(value=0, axis=1)
dta_clean = dta_clean.dropna()
dta_clean = dta_clean.drop('Unnamed: 0', axis=1)
dta_clean.describe()
    
y = dta_clean['worldwide_gross']
X = dta_clean.drop('worldwide_gross', axis=1)

/usr/local/lib/python3.4/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#models = [AdaBoostRegressor,BaggingRegressor,ExtraTreesRegressor,GradientBoostingRegressor,RandomForestRegressor,BayesianRidge,ElasticNet,HuberRegressor,Lars,Lasso,LassoLars,LinearRegression,PassiveAggressiveRegressor,Ridge,SGDRegressor,KNeighborsRegressor,RadiusNeighborsRegressor,MLPRegressor,SVR,LinearSVR,NuSVR,DecisionTreeRegressor,ExtraTreeRegressor]#classifiers = [RadiusNeighborsClassifier]
models= [LinearRegression]
results = {}
errors = {}

def run_grid_search(x,y,Model, res_dict, error):
    #create pipline and use GridSearch to find the betst params
    name = Model.__name__
    if name == "MLPClassifier" or name == "MLPRegressor": 
        cv = 50
    else: 
        cv = 5
    model = Model()
    pipe = Pipeline(steps=[('model', model)])
    #create estimator
    print()
    print ("***Starting ["  + name + "] estimator run ")
    estimator = GridSearchCV(pipe,dict(),verbose=2, cv=cv, n_jobs=4, scoring="adjusted_rand_score")
    #run the esmimator, except eceptions, sape errors
    try:
            estimator.fit(x, y)
            print ("GREP_ME***Results of ["  + name + "] estimator run are")
            print (estimator.cv_results_)
            print ("GREP_ME***Best params of ["  + name + "] estimator run are")
            print (estimator.best_params_)
            print ("GREP_ME***Best score of ["  + name + "] estimator run are")
            print (estimator.best_score_)
            results[name] = estimator.best_score_
    except ValueError as err:
            print ("GREP_ME***Error caught for  ["  + name + "]")
            errors[name] = err
            pass
            
def run_solver(X,y,models_arr, res_dict, errors):
    for model in models_arr:
        run_grid_search(X,y, model, res_dict, errors)    

In [3]:
run_solver(X,y, models, results, errors)


***Starting [LinearRegression] estimator run 
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] ................................................. , total=   0.1s
[CV]  ................................................................
[CV] ................................................. , total=   0.2s
[CV] ................................................. , total=   0.2s
[CV] ................................................. , total=   0.2s
[CV] ................................................. , total=   0.1s
GREP_ME***Results of [LinearRegression] estimator run are
{'std_test_score': array([ 0.]), 'split0_train_score': array([ 0.]), 'split4_test_score': array([ 0.]), 'rank_test_score': arr

[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.4s finished


In [5]:
print(results)

{'BaggingRegressor': -666.72808243134989, 'RandomForestRegressor': -642.91888100286531, 'NuSVR': -4229.7866086802687, 'PassiveAggressiveRegressor': -189.35094285575232, 'KNeighborsRegressor': -681.59015467328902, 'AdaBoostRegressor': -7317.3575743149613, 'LinearSVR': -546.30137487172624, 'Lars': -6.6269030261552457e+46, 'BayesianRidge': -589.57743095752028, 'HuberRegressor': -415.94572792306479, 'ExtraTreesRegressor': -521.51527927181598, 'LinearRegression': -944.8288376262941, 'GradientBoostingRegressor': -1505.8789993953847, 'ElasticNet': -622.71719832057272, 'Lasso': -937.14087446192468, 'Ridge': -931.10037537265475, 'MLPRegressor': -1936627.9124196013, 'ExtraTreeRegressor': -488.3978837298211, 'DecisionTreeRegressor': -2323.9784001044254, 'SGDRegressor': -7.8652893320545996e+34, 'SVR': -1786.2464672394765, 'LassoLars': -923.88815502318494}
